# 1. 环境配置

## 1.1 python 环境准备

In [ ]:
! pip install openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3

## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

# 2. 搜索智能体应用开发

最后除了可以从本地的数据库中获取信息外，还有一个非常重要的信息获取渠道就是网络信息的获取。几乎所有的大模型厂家都添加了网络检索的功能。

那这个功能同样能够在 LangChain 里实现并给到智能体工具进行使用以强化在某些特定场合的实际能力。

在 LangChain 的内置工具里就有大量相关的网络检索工具，包括：
- Bing Search
- Google Search
- SerpAPI
- Tavily Search
- ...

但是 LangChain 中大部分的工具都是需要翻墙的，国内能用的除了 arxiv 和 wikipedia 两个单一数据源搜索是可以使用的外，其他都不能直接使用。
|
所以我们需要使用国内可用的浏览器检索 API 来自行封装，比如博查、百度 AI 搜索和 Jina 等，那下面我们以博查为例来讲解一下如何进行工具的封装。

大家需要自行前往博查的官网（https://open.bochaai.com/）去注册并获取一个 API_KEY 进行使用（可以在购买资源包中购买“免费试用”资源，其他活动可查询文档）。

In [ ]:
from langchain.tools import tool
import requests

BOCHA_API_KEY = "你的博查 api_key"

@tool
def bocha_websearch_tool(query: str, count: int = 10) -> str:
  """
使用Bocha Web Search API 进行搜索互联网网页，输入应为搜索查询字符串，输出将返回搜索结果的详细信息，包括网页标题、网页URL、网页摘要、网站名称、网站Icon、网页发布时间等。
  """
  url = 'https://api.bochaai.com/v1/web-search'
  headers = {'Authorization': f'Bearer {BOCHA_API_KEY}', # 请替换为你的API密钥
    'Content-Type': 'application/json'}
  data = {"query": query,
    "freshness": "noLimit", # 搜索的时间范围，例如 "oneDay", "oneWeek", "oneMonth", "oneYear", "noLimit"
    "summary": True, # 是否返回长文本摘要
    "count": count}
  response = requests.post(url, headers=headers, json=data)
  if response.status_code == 200:
    json_response = response.json()
    try:
      if json_response["code"] != 200 or not json_response["data"]:
        return f"搜索API请求失败，原因是: {response.msg or '未知错误'}"
      webpages = json_response["data"]["webPages"]["value"]
      if not webpages:
        return "未找到相关结果。"
      formatted_results = ""
      for idx, page in enumerate(webpages, start=1):
        formatted_results += (f"引用: {idx}\n"
          f"标题: {page['name']}\n"
          f"URL: {page['url']}\n"
          f"摘要: {page['summary']}\n"
          f"网站名称: {page['siteName']}\n"
          f"网站图标: {page['siteIcon']}\n"
          f"发布时间: {page['dateLastCrawled']}\n\n")
      return formatted_results.strip()
    except Exception as e:
      return f"搜索API请求失败，原因是：搜索结果解析失败 {str(e)}"
  else:
    return f"搜索API请求失败，状态码: {response.status_code}, 错误信息: {response.text}"

在将工具创建好后，我们就可以将其组装到 agent 中进行使用：

In [ ]:
from langchain.agents import create_agent
from langchain_community.chat_models import ChatTongyi

model = ChatTongyi(model="qwen-max")

agent = create_agent(model, tools=[bocha_websearch_tool],
  system_prompt="请你在用户提问后，基于用户提问利用博查检索工具检索后再进行最终的回复。")

然后我们就可以对其进行调用：

In [ ]:
query = "什么是计算机视觉"
for step in agent.stream({"messages": [{"role": "user", "content": query}]},
  stream_mode="values"):
  step["messages"][-1].pretty_print()